In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import theano
import matplotlib
from matplotlib import pyplot
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
import logging 
log = logging.getLogger()
log.setLevel('DEBUG')

Using gpu device 0: GeForce GTX 780 (CNMeM is disabled)


# Experiment Tutorial

In this tutorial, we will run an experiment on the BCI Competition IV dataset. 
We will look at how to run a normal experiment and how to train over smaller windows cut out of the trials.

We have these four steps for the experiment:

1. load the dataset
2. create the layers
3. configure the experiment
4. run the experiment

We assume you have the BCI Competition Data under ```data/bci-competition-iv/2a-combined/```.

## Load the dataset

When loading the dataset, we have the following choiches to make:

* trial window => segment_ival
* what preprocessings to apply to the continuous signal, e.g. resampling, highpass etc.
* what sensors to use (not implemented atm for BCI competition set, since there are only 22 EEG sensors anyways :))

In [3]:
from braindecode.datasets.signal_processor import SignalProcessor
from braindecode.datasets.loaders import BCICompetition4Set2A
from braindecode.datasets.raw import CleanSignalMatrix
from braindecode.datahandling.batch_iteration import WindowsIterator
from braindecode.veganlasagne.monitors import WindowMisclassMonitor
from braindecode.mywyrm.clean import NoCleaner
from braindecode.mywyrm.processing import resample_cnt, highpass_cnt

signal_processor = SignalProcessor(set_loader=BCICompetition4Set2A(
            filename="data/bci-competition-iv/2a-combined/A03TE.mat"),
    segment_ival=[0, 4000], 
    cnt_preprocessors=[[resample_cnt,dict(newfs=150)],
                       [highpass_cnt, dict(low_cut_off_hz=0.5)]])

dataset = CleanSignalMatrix(cleaner=NoCleaner(), 
                                  signal_processor=signal_processor, 
                                  sensor_names='all')
dataset.load()

INFO:braindecode.datasets.raw:Loading set...
INFO:braindecode.datasets.raw:Cleaning set...
INFO:braindecode.datasets.raw:Preprocessing set...
INFO:braindecode.datasets.raw:Loaded clean data with shape (576, 600, 22).
INFO:braindecode.datasets.raw:Loaded dataset with shape: (576, 22, 600, 1)


## Create Layers

Here we will use the raw net, we set the input shape correctly from the loaded dataset. Note that we give names to the layers with weights, we need them later to constrain their norms.

In [4]:
import lasagne
from numpy.random import RandomState
from lasagne.layers import DenseLayer,DropoutLayer,Conv2DLayer, DimshuffleLayer, InputLayer, NonlinearityLayer
from braindecode.veganlasagne.layers import Conv2DAllColsLayer
from braindecode.veganlasagne.pool import SumPool2dLayer
lasagne.random.set_rng(RandomState(9859295))
in_shape = list(dataset.get_topological_view().shape)
in_shape[0] = None # to allow arbitrary sizes of batches
in_layer = InputLayer(shape=in_shape)
network = DimshuffleLayer(incoming= in_layer, pattern=[0, 3, 2, 1])
network = Conv2DLayer(incoming=network, num_filters=40,
    filter_size=[15, 1], nonlinearity=lasagne.nonlinearities.identity, name='time_conv')
network = DropoutLayer(incoming=network, p=0.5)
network = Conv2DAllColsLayer(incoming=network, num_filters=40,
    filter_size=[1, -1], nonlinearity=theano.tensor.sqr, name='spat_conv')
network = SumPool2dLayer(incoming=network, pool_size=[50, 1],
    stride=[10, 1], mode='average_exc_pad')
network = NonlinearityLayer(incoming=network, 
    nonlinearity=theano.tensor.log)
network = DropoutLayer(incoming=network, p=0.5)
network = DenseLayer(incoming=network,
    num_units=4, nonlinearity=lasagne.nonlinearities.softmax, name='final_dense')

## Configure the Experiment

### Dataset Splitting

We first look at the how to split our dataset into train, valid and test set. We split this dataset at a fixed trial, nr. 288, into train and test. Since the dataset has 576 trials, this exactly splits it into half training and half test as in the original BCI Competition.
We use 20% of the **training trials** as our validation set. The validation trials are always at the end of the training set. 

In [5]:
from braindecode.datahandling.splitters import FixedTrialSplitter
dataset_splitter=FixedTrialSplitter(n_train_trials=288, valid_set_fraction=0.2)

### Dataset Preprocessing

We preprocess our dataset along the channel axis. We use online standardization so it could also be applied in the same way for an online experiment.

In [6]:
from braindecode.datahandling.preprocessing import OnlineAxiswiseStandardize
preprocessor = OnlineAxiswiseStandardize(axis=['c', 1]) # the 1 is not important as the 1 dim (form bc01) is empty...

### Batch Iteration

We iterate over balanced batches, so batches are gauranteed to have a maximum size difference of 1. The way this is implemented, batches can get slightly bigger than the given batch size.

In [7]:
from braindecode.datahandling.batch_iteration import BalancedBatchIterator
dataset_iterator = BalancedBatchIterator(batch_size=60)

### Loss and Updates

Our loss is the categorical cross entropy and updates will be computed using Adam.
We will norm constraints, that means the units will be constrained to not have a larger sum of squares than specified.

In [8]:
from braindecode.veganlasagne.update_modifiers import MaxNormConstraint
updates_var_func=lasagne.updates.adam
loss_var_func= lasagne.objectives.categorical_crossentropy

updates_modifier = MaxNormConstraint({
            'time_conv': 2.0,
            'spat_conv': 2.0,
            'final_dense': 0.5})

### Stopping and Monitors

Finally, we have to decide for a stopping and what values to monitor. We monitor loss and misclass (actually we always have to monitor them for the experiment to work) as well as the runtime per epoch.
For stopping, we stop after 10 epochs.

In [9]:
from braindecode.veganlasagne.monitors import LossMonitor, MisclassMonitor, RuntimeMonitor
from braindecode.veganlasagne.stopping import MaxEpochs
from braindecode.experiments.experiment import Experiment
stop_criterion = MaxEpochs(num_epochs=20)
monitors=[LossMonitor(), MisclassMonitor(), RuntimeMonitor()]

exp = Experiment(network, dataset, dataset_splitter, preprocessor,
          dataset_iterator, loss_var_func, updates_var_func, updates_modifier, monitors, stop_criterion)

## Run the Experiment

We setup the experiment and run it.

In [10]:
exp.setup()
exp.run()

INFO:braindecode.experiments.experiment:Layers...
INFO:braindecode.experiments.experiment:InputLayer
INFO:braindecode.experiments.experiment:[None, 22, 600, 1]
INFO:braindecode.experiments.experiment:DimshuffleLayer
INFO:braindecode.experiments.experiment:(None, 1, 600, 22)
INFO:braindecode.experiments.experiment:Conv2DLayer
INFO:braindecode.experiments.experiment:(None, 40, 586, 22)
INFO:braindecode.experiments.experiment:DropoutLayer
INFO:braindecode.experiments.experiment:(None, 40, 586, 22)
INFO:braindecode.experiments.experiment:Conv2DAllColsLayer
INFO:braindecode.experiments.experiment:(None, 40, 586, 1)
INFO:braindecode.experiments.experiment:SumPool2dLayer
INFO:braindecode.experiments.experiment:(None, 40, 54, 1)
INFO:braindecode.experiments.experiment:NonlinearityLayer
INFO:braindecode.experiments.experiment:(None, 40, 54, 1)
INFO:braindecode.experiments.experiment:DropoutLayer
INFO:braindecode.experiments.experiment:(None, 40, 54, 1)
INFO:braindecode.experiments.experiment:De

As you can see from the output, there are two phases.

1. The experiment first runs until the stop criterion is fulfilled (this stop criterion should normally use the performance on the validation set).
2. Then it takes the parameters from the model with the lowest validation misclass. It merges the validation set into the training set and runs until the loss on the validation set is below the loss on the training set at the epoch of the best model. Also it will stop if it has run as many epochs in the second phase as it needed until the lowest validation misclass). This is all hardcoded atm as I am quite happy with this setup :)

In our case, epoch 19 had the lowest validation misclass. In epoch 19, the train loss was 1.91. So it had to reach a validation loss below 1.91 in the second phase, which it did in epoch 23.

## Experiment with windows cut out of trials

We will now use an experiment with windows cut out of trials. First we show the things that remain the same as in the last experiment (except we already change the shape of the input to match our smaller windows).

In [11]:

lasagne.random.set_rng(RandomState(9859295))
in_shape = list(dataset.get_topological_view().shape)
in_shape[0] = None # to allow arbitrary sizes of batches
in_shape[2] = 300 # we will use windows with 300 samples
in_layer = InputLayer(shape=in_shape)
network = DimshuffleLayer(incoming= in_layer, pattern=[0, 3, 2, 1])
network = Conv2DLayer(incoming=network, num_filters=40,
    filter_size=[15, 1], nonlinearity=lasagne.nonlinearities.identity, name='time_conv')
network = DropoutLayer(incoming=network, p=0.5)
network = Conv2DAllColsLayer(incoming=network, num_filters=40,
    filter_size=[1, -1], nonlinearity=theano.tensor.sqr, name='spat_conv')
network = SumPool2dLayer(incoming=network, pool_size=[50, 1],
    stride=[10, 1], mode='average_exc_pad')
network = NonlinearityLayer(incoming=network, 
    nonlinearity=theano.tensor.log)
network = DropoutLayer(incoming=network, p=0.5)
network = DenseLayer(incoming=network,
    num_units=4, nonlinearity=lasagne.nonlinearities.softmax, name='final_dense')

dataset_splitter=FixedTrialSplitter(n_train_trials=288, valid_set_fraction=0.2)
preprocessor = OnlineAxiswiseStandardize(axis=['c', 1])
updates_var_func=lasagne.updates.adam
loss_var_func= lasagne.objectives.categorical_crossentropy
updates_modifier = MaxNormConstraint({
            'time_conv': 2.0,
            'spat_conv': 2.0,
            'final_dense': 0.5})

We use a different iterator which will always cut windows out of the trials. We use windows of 300 samples and a stride of 30 samples between the windows
Also we need to use a different monitor which will take the average over all windows of a trial to compute
the prediction for that trial. This assures we can make a fair comparison between both training ways.
To make our early stopping sensible, we also now stop once the valid misclass has not improved for 2 epochs (normally we would use 20 epochs in this example). 

In [13]:
from braindecode.datahandling.batch_iteration import WindowsIterator
from braindecode.veganlasagne.stopping import Or, NoDecrease
from braindecode.veganlasagne.monitors import WindowMisclassMonitor

dataset_iterator = WindowsIterator(n_samples_per_window=300,batch_size=60,
                                   sample_axes_name=0, n_sample_stride=30)

stop_criterion = NoDecrease('valid_misclass', num_epochs=2)

monitors=[LossMonitor(), WindowMisclassMonitor(), RuntimeMonitor()]

In [14]:
exp = Experiment(network, dataset, dataset_splitter, preprocessor,
          dataset_iterator, loss_var_func, updates_var_func, updates_modifier, monitors, stop_criterion)
exp.setup()
exp.run()

INFO:braindecode.experiments.experiment:Layers...
INFO:braindecode.experiments.experiment:InputLayer
INFO:braindecode.experiments.experiment:[None, 22, 300, 1]
INFO:braindecode.experiments.experiment:DimshuffleLayer
INFO:braindecode.experiments.experiment:(None, 1, 300, 22)
INFO:braindecode.experiments.experiment:Conv2DLayer
INFO:braindecode.experiments.experiment:(None, 40, 286, 22)
INFO:braindecode.experiments.experiment:DropoutLayer
INFO:braindecode.experiments.experiment:(None, 40, 286, 22)
INFO:braindecode.experiments.experiment:Conv2DAllColsLayer
INFO:braindecode.experiments.experiment:(None, 40, 286, 1)
INFO:braindecode.experiments.experiment:SumPool2dLayer
INFO:braindecode.experiments.experiment:(None, 40, 24, 1)
INFO:braindecode.experiments.experiment:NonlinearityLayer
INFO:braindecode.experiments.experiment:(None, 40, 24, 1)
INFO:braindecode.experiments.experiment:DropoutLayer
INFO:braindecode.experiments.experiment:(None, 40, 24, 1)
INFO:braindecode.experiments.experiment:De

The epochs are now much slower, which is because there are much more windows than there were trials before (about 10 times more with our parameters).

## Experiment with model making predictions for every window

One disadvantage of the way to use windows out of the trials like that is net does a lot of computations multiple times. Think of the first convolutional layer over time, when being applied to two neighbouring windows, it will do the same computations multiple times.

A different way to make multiple predictions for one trial is to put it into the model explicitly. If we replace the final dense layer by a convolution over time + average  over all convolution regions, every  convolution region has to have a sensible prediction.

I have never done this before, so let's try it :)
Note the final 3 layers of the network have changed. A filter size of 22 means we are using 22 pooling regions which is again about half of the trial (ignoring border effects).

In [15]:
from lasagne.layers import GlobalPoolLayer
import theano.tensor as T
lasagne.random.set_rng(RandomState(9859295))
in_shape = list(dataset.get_topological_view().shape)
in_shape[0] = None # to allow arbitrary sizes of batches
in_layer = InputLayer(shape=in_shape)
network = DimshuffleLayer(incoming= in_layer, pattern=[0, 3, 2, 1])
network = Conv2DLayer(incoming=network, num_filters=40,
    filter_size=[15, 1], nonlinearity=lasagne.nonlinearities.identity, name='time_conv')
network = DropoutLayer(incoming=network, p=0.5)
network = Conv2DAllColsLayer(incoming=network, num_filters=40,
    filter_size=[1, -1], nonlinearity=theano.tensor.sqr, name='spat_conv')
network = SumPool2dLayer(incoming=network, pool_size=[50, 1],
    stride=[10, 1], mode='average_exc_pad')
network = NonlinearityLayer(incoming=network, 
    nonlinearity=theano.tensor.log)
network = DropoutLayer(incoming=network, p=0.5)
network = Conv2DLayer(incoming=network,
    num_filters=4, filter_size=[22, 1], nonlinearity=lasagne.nonlinearities.identity, name='final_conv')
network = GlobalPoolLayer(incoming=network, pool_function=T.mean)
network = NonlinearityLayer(incoming=network, nonlinearity=lasagne.nonlinearities.softmax) 



dataset_splitter=FixedTrialSplitter(n_train_trials=288, valid_set_fraction=0.2)
preprocessor = OnlineAxiswiseStandardize(axis=['c', 1])
updates_var_func=lasagne.updates.adam
loss_var_func= lasagne.objectives.categorical_crossentropy
updates_modifier = MaxNormConstraint({
            'time_conv': 2.0,
            'spat_conv': 2.0,
            'final_conv': 2.0})
dataset_iterator = BalancedBatchIterator(batch_size=60)

stop_criterion = NoDecrease('valid_misclass', num_epochs=2)

monitors=[LossMonitor(), MisclassMonitor(), RuntimeMonitor()]
exp = Experiment(network, dataset, dataset_splitter, preprocessor,
          dataset_iterator, loss_var_func, updates_var_func, updates_modifier, monitors, stop_criterion)
exp.setup()
exp.run()

INFO:braindecode.experiments.experiment:Layers...
INFO:braindecode.experiments.experiment:InputLayer
INFO:braindecode.experiments.experiment:[None, 22, 600, 1]
INFO:braindecode.experiments.experiment:DimshuffleLayer
INFO:braindecode.experiments.experiment:(None, 1, 600, 22)
INFO:braindecode.experiments.experiment:Conv2DLayer
INFO:braindecode.experiments.experiment:(None, 40, 586, 22)
INFO:braindecode.experiments.experiment:DropoutLayer
INFO:braindecode.experiments.experiment:(None, 40, 586, 22)
INFO:braindecode.experiments.experiment:Conv2DAllColsLayer
INFO:braindecode.experiments.experiment:(None, 40, 586, 1)
INFO:braindecode.experiments.experiment:SumPool2dLayer
INFO:braindecode.experiments.experiment:(None, 40, 54, 1)
INFO:braindecode.experiments.experiment:NonlinearityLayer
INFO:braindecode.experiments.experiment:(None, 40, 54, 1)
INFO:braindecode.experiments.experiment:DropoutLayer
INFO:braindecode.experiments.experiment:(None, 40, 54, 1)
INFO:braindecode.experiments.experiment:Co

We see the speed is much faster but it doesn't seem to learn so much yet, so let's continue training a little longer.

In [16]:
exp.stop_criterion = MaxEpochs(num_epochs=50)
exp.run_until_early_stop()

INFO:braindecode.experiments.experiment:Epoch 0
INFO:braindecode.experiments.experiment:train_loss           2.73759
INFO:braindecode.experiments.experiment:valid_loss           2.71680
INFO:braindecode.experiments.experiment:test_loss            2.78567
INFO:braindecode.experiments.experiment:train_misclass       0.70563
INFO:braindecode.experiments.experiment:valid_misclass       0.63158
INFO:braindecode.experiments.experiment:test_misclass        0.67014
INFO:braindecode.experiments.experiment:runtime              0.00000
INFO:braindecode.experiments.experiment:
INFO:braindecode.experiments.experiment:Time updates this epoch: 0.271549 seconds
INFO:braindecode.experiments.experiment:Epoch 1
INFO:braindecode.experiments.experiment:train_loss           1.44903
INFO:braindecode.experiments.experiment:valid_loss           1.46609
INFO:braindecode.experiments.experiment:test_loss            1.43255
INFO:braindecode.experiments.experiment:train_misclass       0.75325
INFO:braindecode.exper

Ok, better already :) Now I would deem this ready for a more systematic comparison....

## Stuff just for me (Robin) :)

In [1]:
%%capture
import os
import site
site.addsitedir('/home/schirrmr/.local/lib/python2.7/site-packages/')
site.addsitedir('/usr/lib/pymodules/python2.7/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
%cd /home/schirrmr/braindecode/code/braindecode/
assert 'THEANO_FLAGS' in os.environ
# switch to cpu
#os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'